In [58]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime
from osrs_fcns import *
import datetime
import time

In [59]:
headers = {'User-Agent':'GEoutlier-detection'}

In [60]:
api_endpt = "https://prices.runescape.wiki/api/v1/osrs"

In [61]:
items_endpt = "/mapping"
latest_endpt = "/latest"
five_min_endpt = "/5m"
five_min_timestamp = "/5m?timestamp="

In [62]:
items_response = requests.get((api_endpt+five_min_timestamp+'1750188900'), headers=headers)


In [73]:
items_response.text

'{"data":{},"timestamp":1661988900}'

In [63]:
five_min_timestamp = pd.DataFrame.from_dict(json.loads(items_response.text))['timestamp']

In [64]:
five_min_data = pd.DataFrame.from_dict(json.loads(items_response.text)['data']).T

In [65]:
five_min_data['timestamp'] = five_min_timestamp

In [66]:
five_min_data['item_id'] = five_min_data.index

In [75]:
len(five_min_data)

0

In [67]:
five_min_data.reset_index(drop=True)

,timestamp,item_id


In [26]:
five_data = pd.DataFrame(columns=['item_id','timestamp','avgHighPrice','highPriceVolume','avgLowPrice','lowPriceVolume'])

In [27]:
five_data.item_id = five_min_data.item_id
five_data.timestamp = five_min_data.timestamp
five_data.avgHighPrice = five_min_data.avgHighPrice
five_data.highPriceVolume = five_min_data.highPriceVolume
five_data.avgLowPrice = five_min_data.avgLowPrice
five_data.lowPriceVolume = five_min_data.lowPriceVolume


In [28]:
five_data

,item_id,timestamp,avgHighPrice,highPriceVolume,avgLowPrice,lowPriceVolume
2,2,1615188900,161.0,50300.0,159.0,394.0
36,36,1615188900,150.0,2.0,NaN,0.0
45,45,1615188900,NaN,0.0,21.0,15.0
56,56,1615188900,NaN,0.0,34.0,1.0
91,91,1615188900,77.0,37.0,NaN,0.0
...,...,...,...,...,...,...
24613,24613,1615188900,649.0,200.0,NaN,0.0
24621,24621,1615188900,764.0,100.0,NaN,0.0
24635,24635,1615188900,14730.0,30.0,NaN,0.0
24777,24777,1615188900,7544444.0,5.0,NaN,0.0


In [76]:
def get5mItemData(timestamp):
    headers = {'User-Agent':'GEoutlier-detection'}
    api_endpt = "https://prices.runescape.wiki/api/v1/osrs"
    five_min_timestamp = "/5m?timestamp="
    items_response = requests.get((api_endpt+five_min_timestamp+str(timestamp)), headers=headers)
    five_min_timestamp = pd.DataFrame.from_dict(json.loads(items_response.text))['timestamp']
    five_min_data = pd.DataFrame.from_dict(json.loads(items_response.text)['data']).T
    if(len(five_min_data)>0):
        five_min_data['timestamp'] = five_min_timestamp
        five_min_data['item_id'] = five_min_data.index
        five_min_data.reset_index(drop=True,inplace=True)
        five_data = pd.DataFrame(columns=['item_id','timestamp','avgHighPrice','highPriceVolume','avgLowPrice','lowPriceVolume'])
        five_data.item_id = five_min_data.item_id
        five_data.timestamp = five_min_data.timestamp
        five_data.avgHighPrice = five_min_data.avgHighPrice
        five_data.highPriceVolume = five_min_data.highPriceVolume
        five_data.avgLowPrice = five_min_data.avgLowPrice
        five_data.lowPriceVolume = five_min_data.lowPriceVolume
        return five_data
    else:
        return False
    

In [45]:
#write current timestamp to file and pass as a param. look for last timestamp and continue from there iterating
def getLatestTimestamp(file):
    #open file
    file = open(file,'r')
    #read line
    print("Reading New time")
    timeLine = file.readline()
    #close file
    file.close()
    
    return int(timeLine)

In [46]:
def updateTimeFile(newTime,file):
    #open file
    file = open(file,'w')
    #overwrite first line with used timestamp
    print(f'Updating Time File With: {newTime}')
    file.writelines(newTime)
    #close file
    file.close()
    return newTime

In [85]:
def incrementTime(oldTime):
    print('Incrementing Time')
    return oldTime + 300

In [48]:
conn = create_connection('../geitems.db')

2.6.0


In [91]:
def addLatestData(itemdf):
    print('Connecting to Database')
    conn = create_connection('../geitems.db')
    for row in range(0,len(itemdf)):
        item_id = int(itemdf.iloc[row]['item_id'])
        timestamp = int(itemdf.iloc[row]['timestamp'])
        avghigh = float(itemdf.iloc[row]['avgHighPrice'])
        highvol = int(itemdf.iloc[row]['highPriceVolume'])
        avglow = float(itemdf.iloc[row]['avgLowPrice'])
        lowvol = int(itemdf.iloc[row]['lowPriceVolume'])

    
    price = (item_id,timestamp,avghigh,highvol,avglow,lowvol);
    print('Adding Data To Table')
    price_id = add_prices(conn, price)
    print('Disconnecting from Database')
    conn.close()

In [50]:
addLatestData(conn, five_data)

NameError: name 'five_data' is not defined

In [84]:
round(datetime.datetime.timestamp(datetime.datetime.now())*1000)

1661913312237

In [89]:


def main():
    
    
    newTime = getLatestTimestamp('latestTime.txt')
    while(newTime <= round(datetime.datetime.timestamp(datetime.datetime.now()))):
        print('Getting TimeStamp Data')
        newData = get5mItemData(newTime)
        if(len(newData)>0):
            addLatestData(conn, newData)
        
        newTime = incrementTime(newTime)
        
        updateTimeFile(str(newTime),'latestTime.txt')
        time.sleep(np.random.randint(0,3))
    return

In [90]:
main()

Reading New time
Getting TimeStamp Data
Connecting to Database
2.6.0
Adding Data To Table
Disconnecting from Database
Incrementing Time
Updating Time File With: 1615189200
Getting TimeStamp Data
Connecting to Database
2.6.0
Adding Data To Table
Disconnecting from Database
Incrementing Time
Updating Time File With: 1615189500
Getting TimeStamp Data
Connecting to Database
2.6.0
Adding Data To Table
Disconnecting from Database
Incrementing Time
Updating Time File With: 1615189800
Getting TimeStamp Data
Connecting to Database
2.6.0
Adding Data To Table
Disconnecting from Database
Incrementing Time
Updating Time File With: 1615190100
Getting TimeStamp Data
Connecting to Database
2.6.0
Adding Data To Table
Disconnecting from Database
Incrementing Time
Updating Time File With: 1615190400
Getting TimeStamp Data
Connecting to Database
2.6.0
Adding Data To Table
Disconnecting from Database
Incrementing Time
Updating Time File With: 1615190700
Getting TimeStamp Data
Connecting to Database
2.6.0
A

KeyboardInterrupt: 

1661910726527.573

In [ ]:
datetime.datetime.timestamp(datetime.datetime.now())*1000